# Demo: Running Greek Bucketing/Measures using transient portfolio and/or stored portfolio

This notebook demonstrates how to run greek bucketing/measures API using transient portfolio (balances) and/or stored portfolio (portfolio metadata id)

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from serenity_sdk.widgets import ConnectWidget

# if you want to auto-connect, set this environment variable to your desired default
connect_widget = ConnectWidget(os.getenv('SERENITY_CONFIG_ID', None))

In [ ]:
# create an alias to the api
api = connect_widget.get_api()

## Run Greek Measures / Bucketing with transient portfolio

In [ ]:
from serenity_types.portfolio.core import Balance

transient_portfolio = [
    Balance(asset_id="a9bc74b3-c761-4446-b4f2-725ae1dcf4fc", quantity=101748.0),
    Balance(asset_id="78e2e8e2-419d-4515-9b6a-3d5ff1448e89", quantity=1000.0),
    Balance(asset_id="95b89dfd-c5f6-41d6-83c6-a9d97e0ac361", quantity=20000.0),
    Balance(asset_id="5b7256a0-e0a8-4876-a881-6d52db728eaf", quantity=25000.0),
    Balance(asset_id="1b659c93-83b6-4257-8c46-65b5019f0b15", quantity=30000.0),
    Balance(asset_id="be6d3298-cd41-43fe-889a-e7dd37441aec", quantity=35000.0),
    Balance(asset_id="78e2e8e2-419d-4515-9b6a-3d5ff1448e89", quantity=500.0),
    Balance(asset_id="a9bc74b3-c761-4446-b4f2-725ae1dcf4fc", quantity=30000.0)
]

greek_measures_result = api.risk_analytics().compute_greek_measures(pf_balances=transient_portfolio)
greek_measures_result.dict()

## Run Greek Measures / Bucketing with stored portfolios

### (Optional) Get the portfolio metadata id to be used

In [ ]:
# List all of the available portfolio metadatas
metadata_list = api.portfolio().list_portfolio_metadata()

# for demo purpose, pick the first metadata with account (if any) from the available metadata
metadata_with_acc = [m for m in metadata_list if m.account_ids is not None]
metadata_id = metadata_with_acc[0].metadata_id if metadata_with_acc else metadata_list[0].metadata_id
metadata_id

### Run Greek Measures / Bucketing using portfolio metadata id

In [ ]:
greek_measures_result = api.risk_analytics().compute_greek_measures(pf_metadata_id=metadata_id)
greek_measures_result.dict()

## Using different as_of_time

As per API docs, we can specify different "as_of_time" and/or "portfolio_as_of_time"

as_of_time: The as-of time to be used when asking the data source for market data.

portfolio_as_of_time: The as-of time used to determine which portfolio snapshot is to be used.

In [ ]:
from datetime import datetime, timedelta, timezone

# set as_of_time to be 3 days ago i.e. get prices as of 3 days ago.
as_of_time = datetime.now(timezone.utc) - timedelta(days=3)

# set pf_as_of_time to older snapshot datetime
snapshots = api.portfolio().list_portfolio_snapshots(metadata_id)
pf_as_of_time = snapshots[-1].as_of_time

greek_measures_result = api.risk_analytics().compute_greek_measures(
    as_of_time=as_of_time,
    pf_as_of_time=pf_as_of_time,
    pf_metadata_id=metadata_id
)
greek_measures_result.dict()